# Use LLM to extract certain data based on definition

This notebook reads in certain state bills and uses LLM to extract certain data based on some definitions, e.g. whether a bill's scope is on government's use of AI/ADS.

## Import

In [ ]:
import os
import re
import glob
import shutil

import time
import json
import pickle

import numpy as np
import pandas as pd
from tqdm.notebook import tqdm


In [ ]:
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains.llm import LLMChain
from langchain.text_splitter import CharacterTextSplitter, TokenTextSplitter
from langchain.chains.summarize import load_summarize_chain
from langchain.docstore.document import Document

In [ ]:
%load_ext dotenv
%dotenv

## Read and select data

The following selects curated data (i.e. bills that have met certain threshold of AI/ADS-related words, usually relative to the length of the document). However, to widen the search, below also attempts to included bills not marked by such curation based on threshold, as long as they contain `artificial intelligence` or `automat(ed|ic) decision`. Additionally, only **state** bills proposed from 2023-2024 are selected.

In [ ]:
df = pd.read_json('data/bill_data.json')
df['text'] = df['text'].str.lower()


In [ ]:
df['has_ai_ads'] = ~df.filter(regex='^query_').agg(
    lambda x: ' '.join(list(set(np.concatenate(list(x))))),
    axis=1
).str.extract(
    r'(artificial intelligence|automat\w* decision)',
    expand=False
).isna()

In [ ]:
df = (
    df
    .query(
        '''
        (curated or has_ai_ads)
        and
        first_date.str.contains("2023|2024", regex=True)
        and 
        jurisdiction_code != "US"
    '''.replace('\n',' ').strip()
    )
    .reset_index(drop=True)
)

len(df)

## Split text

Some bills are quite long, so break into chunks

In [ ]:
text_splitter = TokenTextSplitter(
    chunk_size=10000,
    chunk_overlap=200,
)

In [ ]:
docs = []

for _, row in df.iterrows():
    text = row['text']
    metadata = {
        k: v for k, v in row.items() 
        if k not in ['text', 'raw_text', 'abstract']
    }
    
    chunks = text_splitter.split_documents([Document(page_content=text)])
    docs.append(dict(
        bill_id = metadata['bill_id'],
        chunks = chunks,
        metadata = metadata,
        num_chunks = len(chunks)
    ))


In [ ]:
len(docs), sum([len(x['chunks']) for x in docs])

## Definitions

These are some definitions and intructions to handle them. 

In [ ]:
definitions = [
    dict(
        name = "has_government_scope",
        description = '''\
Indicates whether the bill has government scope: \
a bill has government scope if it governs the government's use of artificial intelligence (AI) or \
automated decision systems (ADS) in its operations. \
This scope specifically focuses on the **government**'s use and procurement of these technologies.''',
        instruction = '''\
- First, answer only "Yes" or "No".
- If "Yes", also include 1-2 sentence excerpts from the text to support the government scope, \
label variable as `excerpt_government_scope`.'''
    ),
    dict(
        name = "has_ai_governance_body",
        description = '''\
Indicates whether the bill designates, indicates or establishes an AI governance body: \
an AI governance body is a group of people in the within a government entity or organization that \
has the authority to manage and oversee the use of AI or ADS by that entity or organization.''',
        instruction = '''\
- First, answer only "Yes" or "No".
- If "Yes", also include the name(s) of the governance body, \
label variable as `ai_governance_body_names`.'''
    ),
    dict(
        name = "has_harmonization",
        description = '''\
Indicates whether the bill outlines intent or strategy to harmonize legislation between state and federal government. \
Harmonization is defined as cooperation between different state and federal jurisdictions \
to make laws identical or at least more similar.''',
        instruction = '''\
- First, answer only "Yes" or "No".
- If "Yes", also include 1-2 sentence excerpts from the text to support existence of hamornization, \
label variable as `excerpt_harmonization`.'''
    ),
]

definitions

In [ ]:
def_str = '\n'.join([
'''\
{index}. `{name}`: {description}

Instruction: 
{instruction}
'''.format(**d, index=i+1)
for i, d in enumerate(definitions)
])

print(def_str)

## Create LLM chain

Note: Technically there are probably better ways to do this, like with structured data extraction & `pydandic`.

The following uses summary chain with refinement instead as some documents are long and broken into chunks.

In [ ]:
# note: more models can be input here if need be
llms = [
    ChatOpenAI(
        temperature=0,
        model_name="gpt-3.5-turbo-0125",
    ),
]

In [ ]:
prompt_template = """You are a helpful assistant for legislators, researchers and lawyers.
You are given a task to read a bill and extract necessary information from them.
Below are the variables and instructions:

%s 

Use only the definitions and follow instructions here.
Only use the existing text as reference. Do not make things up.
If you know an answer is empty, just use an empty string "". 
If you do not know an answer for a variable, just answer as "unknown".

Please output as a JSON format.

Here is the text:

{text}

JSON_OUTPUT:""" %(def_str)
prompt = PromptTemplate.from_template(prompt_template)

refine_template = '''
Your job is extract necessary information about a bill.
We have provided an existing JSON output up to a certain point: 

```json
{existing_answer}
```

We have the opportunity to refine the existing JSON output\
(only if needed) with some more context below.
------------
{text}
------------
Given the new context, refine the original JSON output.
If the context isn't useful, return the original JSON output.

The final answer should be a JSON output with only these variables:

%s
''' %(def_str)

refine_prompt = PromptTemplate.from_template(refine_template)


chains = []
for llm in llms:
    if hasattr(llm, 'model'):
        model_source = llm.model
    if hasattr(llm, 'model_name'):
        model_source = llm.model_name
    chains.append(dict(
        model = model_source,
        chain = load_summarize_chain(
            llm=llm,
            chain_type="refine",
            question_prompt=prompt,
            refine_prompt=refine_prompt,
            return_intermediate_steps=True,
            input_key="input_documents",
            output_key="output_text",
        )
    ))


## Extract from chain

In [ ]:
outputs = dict()

for i, bill in tqdm(enumerate(docs), total = len(docs)):
    bill_id = bill['bill_id']
    for chain_dict in chains:
        model = chain_dict['model']
        chain = chain_dict['chain']
        
        outkey = (model, bill_id)
        if outkey in outputs:
            continue
        
        result = chain({"input_documents": bill['chunks']})
        outputs[outkey] = dict(
            model = model,
            bill_id = bill_id,
            extraction = result['output_text'],
            metadata = bill['metadata']
        )


In [ ]:
for k, v in outputs.items():
    assert k == (v['model'], v['bill_id'])

## Process output

In [ ]:
dfe = pd.DataFrame([dict(
    bill_id = x['bill_id'],
    model = x['model'],
    **{
        k: v for k, v in x['metadata'].items()
        if k in  [
            'session', 
            'classification',
            'jurisdiction',
            'jurisdiction_code',
            'introduced_chamber',
            'first_date',
            'title',
            'curated',
            'has_ai_ads'
        ]
    },
    **json.loads(
        re.search(r'{((.|\n)+)}', x['extraction']).group()
    )
) for x in outputs.values()])

In [ ]:
for k in dfe.filter(regex='has_|is_').columns:
    if dfe[k].dtype == bool:
        continue
    dfe[k] = (
        dfe[k].str.upper()
        .fillna('NO')
        .map({
            '': False, 
            'NO': False, 
            'YES': True
        })
    )


dfe['ai_governance_body_names'] = dfe['ai_governance_body_names'].fillna('').apply(
    lambda x: x.title() if isinstance(x, str) else '; '.join(x).title()
)

dfe['excerpt_government_scope'] = dfe['excerpt_government_scope'].fillna('')
dfe['excerpt_harmonization'] = dfe['excerpt_harmonization'].fillna('')

assert all(dfe['has_government_scope'] == (dfe['excerpt_government_scope'] != ""))
assert all(dfe['has_ai_governance_body'] == (dfe['ai_governance_body_names'] != ""))
assert all(dfe['has_harmonization'] == (dfe['excerpt_harmonization'] != ""))

In [ ]:
dfe

In [ ]:
print(
    dfe['has_ai_governance_body'].unique(),
    dfe['has_government_scope'].unique(),
    dfe['has_harmonization'].unique()    
)

In [ ]:
dfe.to_csv('data/memo-criteria-extract.csv', index=False)